In [ ]:
import os
os.environ["GROQ_API_KEY"] = ""  # 🗝️ Replace with your Groq key

In [2]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool, InjectedToolArg
from typing import Annotated
import requests

In [3]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """Fetch conversion rate between base and target currency"""
    url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'
    response = requests.get(url)
    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """Convert base_currency_value to target using conversion_rate"""
    return base_currency_value * conversion_rate

In [8]:
llm = ChatGroq(
    model="qwen-qwq-32b",  # or llama3-70b-8192
    temperature=0.7
)

In [9]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [10]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="What is the conversion rate from INR to USD and convert 10 INR to USD")]

In [11]:
ai_message = llm_with_tools.invoke(messages)

In [12]:
import json
conversion_rate = None

for tool_call in ai_message.tool_calls:
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        messages.append(tool_message1)
    if tool_call['name'] == 'convert':
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)

In [13]:
llm_with_tools.invoke(messages).content

'The conversion rate from INR to USD is **0.01165** (1 INR = 0.01165 USD).  \n\nFor **10 INR** to USD:  \n10 INR × 0.01165 = **0.1165 USD**.  \n\nSo, 10 INR equals approximately **0.1165 USD**.'